In [16]:
import pandas as pd
import numpy as np
from sklearn import metrics, preprocessing, linear_model
import matplotlib.pyplot as plt
colors = ["#476A2A", "#7851B8", "#BD3430", "#4A2D4E", "#875525",
          "#A83683", "#4E655E", "#853541", "#3A3120", "#535D8E"]

In [2]:
np.random.seed(0)

print("Loading Data...")
training_data = pd.read_csv("numerai_training_data.csv", header=0)
prediction_data = pd.read_csv("numerai_tournament_data.csv", header=0)

print(training_data.shape)
print(prediction_data.shape)

Loading Data...
(393613, 54)
(243377, 54)


In [5]:
'''
전처리
'''
features = [f for f in list(training_data) if "feature" in f]
# print(features[:5])
X = training_data[features]
# print(X.shape)
Y = training_data["target"]
# print(Y.shape)
x_prediction = prediction_data[features]
# print(x_prediction.shape)
ids = prediction_data["id"]
# print(ids.shape)

In [6]:
#'''분석'''
model = linear_model.LogisticRegression(n_jobs=-1) #모든 CPU 코어 활용
model.fit(X, Y)

y_prediction = model.predict_proba(x_prediction) 
# '''
#     predict_proba
#     0으로 예측하는 확신의 정도
#     1로 예측하는 확신의 정도
# '''
results = y_prediction[:,1] #'''1로 예측한 확률만 선택'''

/Users/dohk/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


In [7]:
'''예측 결과'''
y_pred = model.predict(x_prediction)
print(y_pred[:10])

[1 0 0 1 1 1 1 0 0 1]


In [8]:
print(y_prediction[:10])

[[0.47319235 0.52680765]
 [0.50897178 0.49102822]
 [0.50918814 0.49081186]
 [0.46976612 0.53023388]
 [0.4643938  0.5356062 ]
 [0.46571597 0.53428403]
 [0.48067065 0.51932935]
 [0.5390022  0.4609978 ]
 [0.50043374 0.49956626]
 [0.45939698 0.54060302]]


In [9]:
print(results[:10]) '''id 별 probability. 제출할 내용'''

SyntaxError: invalid syntax (<ipython-input-9-9b5f60d5c6aa>, line 1)

In [ ]:
results_df = pd.DataFrame(data={'probability':results})
joined = pd.DataFrame(ids).join(results_df)

In [ ]:
joined.to_csv("predictions.csv", index=False)

# logloss값

http://fastml.com/revisiting-numerai/
<img src="http://wiki.fast.ai/images/math/a/4/6/a4651d4ad311666c617d57c1dde37b28.png"></img>
http://wiki.fast.ai/index.php/Log_Loss
binary classification이고, y가 0이나 1 둘 중의 하나의 값을 갖는 게 맞음
1만 보니까, 20만개의 로그로스에 대한 합을 평균낸 결과가 사이트에 뜨는 logloss값

정의에 의하면 0일 땐 probability값이 작아야 하고
1일 땐 값이 커야 logloss값이 높아지게 된다.





# Originality

<img src="http://fastml.com/images/numerai/2017/two_sample_kolmogorov_smirnov_test.png"></img>

각 확률 구간별 빈도를 구하여 히스토그램을 구하고 Density Function을 통해 CDF를 만든다. 누적..

다른 사람과의 차이를 구하여 가장 큰 차이값을 보이는 값이 threshold값 이상이어야 통과
즉 다른이의 값의 분포가 차이가 나야 함

# consistency

일관성


era별 prob 분포에 차이가 없어야 함


# concordance

조작 없어야 하고

하나의 모델에서 결과물이 나와야

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(learning_rate=0.01, random_state=1, max_depth=1, n_estimators=10) 
model.fit(X, Y)

y_prediction = model.predict_proba(x_prediction) 
results = y_prediction[:,1]

In [11]:
results_df = pd.DataFrame(data={'probability':results})
joined = pd.DataFrame(ids).join(results_df)
joined.to_csv("predictions_gbrt.csv", index=False)

In [12]:
from sklearn.manifold import TSNE
tsne = TSNE(random_state=42) # fitting과 transform 동시에 해줌
# use fit_transform instead of fit, as TSNE has no transform method
digits_tsne = tsne.fit_transform(X[:10])

from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(learning_rate=0.01, random_state=1, max_depth=1, n_estimators=10) 
model.fit(X, Y)

y_prediction = model.predict_proba(x_prediction) 
results = y_prediction[:,1]
results_df = pd.DataFrame(data={'probability':results})
joined = pd.DataFrame(ids).join(results_df)
joined.to_csv("predictions_gbrt.csv", index=False)

In [18]:
plt.figure(figsize=(10, 10))
plt.xlim(digits_tsne[:, 0].min(), digits_tsne[:, 0].max() + 1)
plt.ylim(digits_tsne[:, 1].min(), digits_tsne[:, 1].max() + 1)
for i in range(len(X)):
    # actually plot the digits as text instead of using scatter
    plt.text(digits_tsne[i, 0], digits_tsne[i, 1], str(Y[i]),
             color = colors[Y[i]],
             fontdict={'weight': 'bold', 'size': 9})
plt.xlabel("t-SNE feature 0")
plt.xlabel("t-SNE feature 1")

IndexError: index 10 is out of bounds for axis 0 with size 10